# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [78]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
import sys
sys.path.insert(0, 'C:/Users/geral/OneDrive/Documents/GitHub/python-api-challenge')
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [79]:
cities_df = pd.read_csv('C:/Users/geral/OneDrive/Documents/GitHub/python-api-challenge/WeatherPy/Resources/cities.csv', index_col=[0])
cities_df.dropna(inplace = True) 
cities_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Alice Springs,-23.7000,133.8833,83.75,24,75,13.80,AU,1669617566
1,Jumla,29.2747,82.1838,64.49,21,0,6.17,NP,1669617544
2,Tazovsky,67.4667,78.7000,9.86,95,100,10.20,RU,1669617567
3,Busselton,-33.6500,115.3333,75.92,43,0,13.67,AU,1669617568
4,Bluff,-46.6000,168.3333,59.81,71,100,9.26,NZ,1669617568
...,...,...,...,...,...,...,...,...,...
557,Kīhei,20.7850,-156.4656,74.84,86,40,3.44,US,1669617866
558,Leh,34.1667,77.5833,24.84,45,0,1.54,IN,1669617866
559,Mayskiy,47.6931,40.1025,33.21,72,100,11.01,RU,1669617867
560,Sai Buri,6.7013,101.6168,85.71,71,41,7.74,TH,1669617868


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [121]:
#configure gmaps
gmaps.configure(api_key=g_key)

# Store latitude and longitude in locations
locations = cities_df[["Lat", "Lng"]]

# Convert humidity to float
humidity = cities_df["Humidity"].astype(float)

# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, 
                                 max_intensity=10, point_radius=1)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [123]:
#create new dataframe with cities that have my ideal weather conditions

#maximum temperature values will be lower than 82 F and higher than 69 F,
#humidity will be lower than 70%, cloudiness of 0%, wind speed lower than 5 mph
ideal_weather = cities_df.loc[(cities_df['Max Temp']<82) & 
                              (cities_df['Max Temp'] >69) &
                              (cities_df['Humidity']<70) &
                              (cities_df['Cloudiness']==0) &
                              (cities_df['Wind Speed']<5)
                            ]

#drop rows with null values
ideal_weather = ideal_weather.dropna()

ideal_weather

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
173,Kiraoli,27.1500,77.7833,77.49,23,0,1.88,IN,1669617663
239,Tura,25.5198,90.2201,77.65,39,0,2.33,IN,1669617701
242,San Miguel de Tucumán,-26.8241,-65.2226,69.80,60,0,4.61,AR,1669617703
262,Tīkamgarh,24.7450,78.8333,77.25,21,0,3.87,IN,1669617714
320,Mīrābād,30.4362,61.8383,74.19,14,0,1.99,AF,1669617746
321,Cabo San Lucas,22.8909,-109.9124,71.51,64,0,3.00,MX,1669617632
430,Baikunthpur,23.2500,82.5500,76.51,29,0,1.79,IN,1669617797
474,Chūru,28.3000,74.9500,78.28,13,0,1.72,IN,1669617737
551,Baghmara,23.7950,86.2085,79.65,28,0,4.56,IN,1669617864


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [108]:
#store ideal_weather in hotel_df, add hotel name column
hotel_df = pd.DataFrame(ideal_weather, columns=['City', 'Country', 'Lat', 'Lng', 'Hotel Name'])

hotel_df

,City,Country,Lat,Lng,Hotel Name
173,Kiraoli,IN,27.1500,77.7833,NaN
239,Tura,IN,25.5198,90.2201,NaN
242,San Miguel de Tucumán,AR,-26.8241,-65.2226,NaN
262,Tīkamgarh,IN,24.7450,78.8333,NaN
320,Mīrābād,AF,30.4362,61.8383,NaN
321,Cabo San Lucas,MX,22.8909,-109.9124,NaN
430,Baikunthpur,IN,23.2500,82.5500,NaN
474,Chūru,IN,28.3000,74.9500,NaN
551,Baghmara,IN,23.7950,86.2085,NaN


In [113]:
#set parameters to search hotels within 5000 meters from each city
params = {
    'type': 'lodging',
    'radius': 5000,
    'key': g_key
}

#iterate through rows of dataframe
for index, row in hotel_df.iterrows():
    #set the 'Lat' column as the latitude
    lat = row['Lat']
    #set the 'Lng' column as the longitude
    lng = row['Lng']
    #set the location parameter as the latitude and longitude in each row
    params['location'] = f"{lat},{lng}"
    #introduce url to search hotels
    base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'
    #save the hotel information
    hotel_name = requests.get(base_url,params=params).json()
    try:
        #store the hotel name in the datafram
        hotel_df.loc[index, 'Hotel Name'] = hotel_name['results'][0]['name']
    except (IndexError):
        #no hotels were found, store as ' ' in dataframe
        hotel_df.drop(index, inplace=True)
        
#drop rows with null values
hotel_df = hotel_df.dropna()
hotel_df
hotel_df


,City,Country,Lat,Lng,Hotel Name
173,Kiraoli,IN,27.1500,77.7833,Taj Hotel
239,Tura,IN,25.5198,90.2201,Hotel RIKMAN Continental
242,San Miguel de Tucumán,AR,-26.8241,-65.2226,Hotel Premier
262,Tīkamgarh,IN,24.7450,78.8333,White House
321,Cabo San Lucas,MX,22.8909,-109.9124,Hotel Tesoro Los Cabos
430,Baikunthpur,IN,23.2500,82.5500,Hotel Shree Annapurna
474,Chūru,IN,28.3000,74.9500,Malji Ka Kamra
551,Baghmara,IN,23.7950,86.2085,Shankar tea shop


In [114]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
#create marker layer
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

#add marker layer
fig.add_layer(marker_layer)

#display figure showing hotels
fig